In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from utils.df_loader import load_diabetes_df
from utils.preprocessing import preprocess_df
from sklearn.model_selection import train_test_split
from utils.dice import generate_dice_result, process_results
from utils.models import train_three_models, evaluation_test, save_three_models, load_three_models
from utils.save import save_result_as_csv
from utils.gs import generate_gs_result, process_results


pd.options.mode.chained_assignment = None 

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)


TF version:  2.4.0-rc0
Eager execution enabled:  True


In [2]:
#### Select dataset ####

dataset_name = 'diabetes' 

if  dataset_name == 'diabetes':
    dataset_loading_fn = load_diabetes_df
else:
    raise Exception("Unsupported dataset")

In [3]:
#### Load datafram info.
df_info = preprocess_df(dataset_loading_fn)

In [4]:
### Seperate to train and test set.
train_df, test_df = train_test_split(df_info.dummy_df, train_size=.8, random_state=seed, shuffle=True)

In [5]:
### Get training and testing array.
X_train = np.array(train_df[df_info.ohe_feature_names])
y_train = np.array(train_df[df_info.target_name])
X_test = np.array(test_df[df_info.ohe_feature_names])
y_test = np.array(test_df[df_info.target_name])

In [6]:
### Load models.
models = load_three_models(X_train.shape[-1], dataset_name)

In [7]:
### Print out accuracy on testset.
evaluation_test(models, X_test, y_test)

DT: [0.7727] | RF [0.7987] | NN [0.7662]


In [8]:
### Setting up the CF generating amount.
num_instances = 5
num_cf_per_instance = 1

In [9]:
results = generate_gs_result(df_info, test_df, models, num_instances, num_cf_per_instance, 2000)
result_dfs = process_results(df_info, results)

Finding counterfactual for dt
instance 0
CF 0
506 ennemies found in initial sphere. Zooming in...
0 ennemies found in initial sphere. Zooming in...
Exploring...
Final number of iterations:  11
Final radius:  (0.019, 0.0208)
Final number of ennemies:  4
Feature selection...
Reduced 7 coordinates
instance 1
CF 0
1207 ennemies found in initial sphere. Zooming in...
2 ennemies found in initial sphere. Zooming in...
0 ennemies found in initial sphere. Zooming in...
Exploring...
Final number of iterations:  49
Final radius:  (0.00873999999999999, 0.00891999999999999)
Final number of ennemies:  1
Feature selection...
Reduced 7 coordinates
instance 2
CF 0
513 ennemies found in initial sphere. Zooming in...
0 ennemies found in initial sphere. Zooming in...
Exploring...
Final number of iterations:  16
Final radius:  (0.027999999999999997, 0.029799999999999997)
Final number of ennemies:  3
Feature selection...
Reduced 7 coordinates
instance 3
CF 0
50 ennemies found in initial sphere. Zooming in..

In [10]:
### Save result as file.
save_result_as_csv("GS", dataset_name, result_dfs)

Result has been saved to ./results/GS_diabetes
